<a href="https://colab.research.google.com/github/avleenk2312/MATH313/blob/main/StochasticRounding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

# Floating-Point Arithmetic

The default floating-point number system is **Float64**, which is double precision on **MATLAB**. Let us find the value of machine epsilon and range as follows.

In [ ]:
eps()

In [ ]:
floatmax()

In [ ]:
floatmin()

# Basics of Rounding

## Types of functions:

[1] **ceil(x)** returns the next largest integer to x (towards + ∞).

[2] **floor(x)** returns the next smallest integer to x (towards - ∞).

[3] **trunc(x)** returns the closest integer towards zero (round to zero).

[4] **round(x,mode)** function with several modes options, see: https://docs.julialang.org/en/v1/base/math/#Base.round-Tuple{Type,%20Any}


In [ ]:
ceil(1.3)

In [ ]:
floor(1.3)

In [ ]:
trunc(1.3)

In [ ]:
ceil(-1.3)

In [ ]:
floor(-1.3)

In [ ]:
trunc(-1.3)

In [ ]:
round(1.3) ## round to nearest by default

In [ ]:
round(-1.3) ## round to neart by deafult

In [ ]:
round(1.3,RoundToZero)

In [ ]:
round(-1.3,RoundToZero)

In [ ]:
round(1.3,RoundUp)

In [ ]:
round(1.3,RoundDown)

In [ ]:
round(-1.3,RoundUp)

In [ ]:
round(-1.3,RoundDown)

# Stochastic Rounding

Not included in Julia by deafult, a package is available called StochasticRounding.jl

I used a popular randomly round to nearest integer idea as formed in  https://gist.github.com/coleoguy/a50dba496fc55440cb4945fe8e25d703 as mentioned in https://medium.com/@minghz42/what-is-stochastic-rounding-b78670d0c4a to create a version for it in Julia.

In [ ]:
using Pkg
Pkg.add("Distributions")

In [ ]:
using Distributions
function StochasticRound(x)
    # extract the decimal portion
    r=trunc(x)
    q=abs(x-r)
    # draw a value 0 or 1 with probability
    # based on how close we already are
    Δx = wsample(0:1,[1-q,q],1)
     # make it negative if x is negative
     if(x<0)
         Δx=-Δx
     end
     # return our new value
     r+Δx[1]
end

In [ ]:
StochasticRound(1.3)

In [ ]:
StochasticRound(1.3)

## How to "see" the probability?

Let us plot it! 



In [ ]:
using Plots

global ℘=0
n=10000
print("Enter x:")
x = readline()
x=parse(Float64,x)
for i=1:n
    result=StochasticRound(x)
    if result==ceil(x)
        global ℘=℘+1
    end
end
global ℘=℘/n
Plots.gr() # the default GR backend look for pie charts
x = ["ceil(x)", "floor(x)"]
y = [℘,1-℘]
pie(x, y, title="Stochastic Rounding", l=0.5)